In [2]:
from __future__ import division
from __future__ import print_function
import sys

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

from pygcn.utils import load_data, accuracy
from pygcn.models import GCN

# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=400,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=16,
                    help='Number of hidden units.')
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')

args, unknown = parser.parse_known_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [3]:
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...


In [4]:
adj.shape

torch.Size([2708, 2708])

In [5]:
idx_train.shape

torch.Size([140])

In [6]:
idx_val.shape

torch.Size([300])

In [7]:
idx_test.shape

torch.Size([1000])

In [9]:
model = GCN(nfeat=features.shape[1],
            nhid=args.hidden,
            nclass=labels.max().item() + 1,
            dropout=args.dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=args.lr, weight_decay=args.weight_decay)

if args.cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

In [10]:
model

GCN(
  (gc1): GraphConvolution (1433 -> 16)
  (gc4): GraphConvolution (16 -> 7)
)

In [11]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Training-validation  results:'
          'Epoch: {:04d}'.format(epoch+1),
          
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

In [12]:
def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))


# Train model
t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

Training-validation  results:Epoch: 0001 loss_train: 1.9012 acc_train: 0.2786 loss_val: 1.8797 acc_val: 0.3500 time: 3.6613s
Training-validation  results:Epoch: 0002 loss_train: 1.8928 acc_train: 0.2929 loss_val: 1.8723 acc_val: 0.3500 time: 0.0058s
Training-validation  results:Epoch: 0003 loss_train: 1.8885 acc_train: 0.2929 loss_val: 1.8652 acc_val: 0.3500 time: 0.0057s
Training-validation  results:Epoch: 0004 loss_train: 1.8688 acc_train: 0.2929 loss_val: 1.8587 acc_val: 0.3500 time: 0.0053s
Training-validation  results:Epoch: 0005 loss_train: 1.8711 acc_train: 0.2929 loss_val: 1.8526 acc_val: 0.3500 time: 0.0053s
Training-validation  results:Epoch: 0006 loss_train: 1.8579 acc_train: 0.2929 loss_val: 1.8469 acc_val: 0.3500 time: 0.0053s
Training-validation  results:Epoch: 0007 loss_train: 1.8573 acc_train: 0.2929 loss_val: 1.8415 acc_val: 0.3500 time: 0.0054s
Training-validation  results:Epoch: 0008 loss_train: 1.8438 acc_train: 0.2929 loss_val: 1.8362 acc_val: 0.3500 time: 0.0054s


#Conclusion: 
1- You entered all features during training but calculated loss between only 140 samples (Labeled).

2- claculated the accuracy for training (140 examples). 

3- calculated  accuracy and loss for Validation but do not. 
backpropagate (300 examples).

4- calculated  accuracy and loss for Testing but do not backpropagate (1000 examples).